In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models
import sinabs
from torchvision import datasets, transforms
from PIL import Image
import sinabs.layers as sl
import numpy as np
import quartz
import copy
from tqdm.auto import tqdm
from quartz.utils import get_accuracy, encode_inputs, decode_outputs, plot_output_histograms, plot_output_comparison, normalize_weights, count_n_neurons, fuse_all_conv_bn
from typing import List

np.set_printoptions(suppress=True)


In [ ]:
from cifar10_models.mobilenetv2 import mobilenet_v2

model = mobilenet_v2(pretrained=True)
model = nn.Sequential(*model.features, nn.AvgPool2d(4), nn.Flatten(), *model.classifier)
model.eval();

In [ ]:
batch_size = 128
device = 'cuda'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616))
])

valid_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(dataset=valid_dataset, batch_size=100, shuffle=True)

In [ ]:
get_accuracy(model, valid_loader, device)

In [ ]:
folded_model = copy.deepcopy(model)
fuse_all_conv_bn(folded_model)

In [ ]:
get_accuracy(folded_model, valid_loader, device)

In [ ]:
folded_model